In [2]:
# Hello!
# Please run this cell either with Shift + Enter or by using Cell -> Run Cells from the taskbar uptop
# Afterwards you will be prompted to enter an initial configuration file, please ensure that you know the path for your
# desired input file.
# Thank you!

import ipywidgets as widgets
from ipywidgets import Box, Layout, HBox

%matplotlib ipympl

import sys
import os
import importlib
import MASTML
from MASTMLInitializer import ConfigFileConstructor

home = os.getcwd()

def VBox(*pargs, **kwargs):
    """Displays multiple widgets vertically using the flexible box model."""
    box = Box(*pargs, **kwargs)
    box.layout.display = 'flex'
    box.layout.flex_flow = 'column'
    box.layout.align_items = 'stretch'
    box.layout.border = 'solid'
    return box

def arrayifyBox(Box):
    out = []
    for i in range(len(Box.children)):
        state = Box.children[i].get_state()
        if 'BoxModel' not in state['_model_name']: #child is a widget
            try:
                if state['description'] == "field": #chlid is a label of a field
                    temp = str(state['value']) + "="
                else: #child is a widget with a value
                    if state['_model_name'] == 'SelectMultipleModel':
                        tup = Box.children[i].value
                        temp = ' ' + ','.join(tup)
                    else:
                        temp = ' ' + str(Box.children[i].value)
                out.append(temp.rstrip())
            except: KeyError
        else: #child is a box
            out.append(arrayifyBox(Box.children[i])) #grab the widgets from the child
    return out

def exportBox(Box, filename):
    arr = arrayifyBox(Box)
    file = open(filename, "w")
    for lines in arr:
        for line in lines:
            file.write(line)
        file.write("\n")
    file.close()
    
def importArray(filename):
    file = open (filename, "r")
    arr = []
    for line in file.readlines():
        if (line.rstrip() != ''):
            arr.append(line.rstrip())
    file.close()
    return arr

def typeWidgets(arr):
    types = []
    for i in range(len(arr)):
        split = arr[i].split('=')
        if(len(split)==1): #element is a label
            types.append("Label")
        else:
            val = split[1].strip() #fetch the value
            try:
                x = float(val)
                if x % 1 == 0:
                    types.append("Int")
                else:
                    types.append("Float")
            except ValueError:
                if val == "True" or val == "False":
                    types.append("Boolean")
                else:
                    types.append("String")
    return types

widgLayout = Layout(width = '45%', height = '40px',font_size='12px')
def makeStringWidget(elem,template,tests):
    split = elem.split('=')
    desc = split[0]
    val = split[1]
    if(desc.strip() == 'feature_selection_algorithm'):
            opts = template['Feature Selection']['feature_selection_algorithm']
            widg = makeDropdown(desc,val,opts)
    elif(desc.strip() == 'models'):
            opts = template['Models and Tests to Run']['models']
            widg = makeDropdown(desc,val,opts)
    elif(desc.strip() == 'test_cases'):
            opts = tests
            widg = makeSelectMutli(desc,[val],opts)
    elif(desc.strip() == 'model_to_use_for_learning_curve'):
            opts = template['Feature Selection']['model_to_use_for_learning_curve']
            widg = makeDropdown(desc,val,opts)
    elif(desc.strip() == 'scoring_metric'):
            opts = template['Feature Selection']['scoring_metric']
            widg = makeDropdown(desc,val,opts)
    else:
        widg = HBox([widgets.Label(value=desc, description = 'field', layout = widgLayout),
                 widgets.Text(
            value=val,
            placeholder = '',
            description = '',
            layout = widgLayout,
            disabled = False
        )])
    return widg

def makeFloatWidget(elem):
    split = elem.split('=')
    desc = split[0]
    val = split[1]
    widg = HBox([widgets.Label(value=desc, description = 'field',layout = widgLayout),
                 widgets.FloatText(
            value=val,
            placeholder = '',
            description = '',
            layout = widgLayout,
            disabled = False
    )])
    return widg

def makeIntWidget(elem):
    split = elem.split('=')
    desc = split[0]
    val = split[1]
    widg = HBox([widgets.Label(value=desc, description = 'field',layout = widgLayout),
                 widgets.IntText(
            value=val,
            placeholder = '',
            description = '',
            layout = widgLayout,
            disabled = False
    )])
    return widg

def makeBoolWidget(elem):
    split = elem.split('=')
    desc = split[0]
    val = split[1].strip()
    widg = HBox([widgets.Label(value=desc, description = 'field',layout = widgLayout),
                 widgets.Dropdown(
        options=['True','False'],
        value = val,
        description = '',
        layout = widgLayout,
        disabled=False
    )])
    return widg

def makeDropdown(desc,val,opts):
    widg = HBox([widgets.Label(value=desc,description = 'field',layout = widgLayout),
                    widgets.Dropdown(
                options = opts,
                value = val.strip(),
                description = '',
                layout=widgLayout,
                disabled = False
            )])
    return widg

def makeSelectMutli(desc,val,opts):
    for i in range(len(val)):
        val[i] = val[i].strip()
    widg = HBox([widgets.Label(value=desc,description = 'field',layout = widgLayout),
                    widgets.SelectMultiple(
                options = opts,
                value = val,
                description = '',
                width = '45%', 
                height = '100px',
                font_size='12px',
                disabled = False
            )])
    return widg

def getTests(arr,template):
    possibles = []
    for i in range(len(arr)):
        desc = arr[i].split('=')[0].strip()
        if(desc[0]=='[' and desc[1]=='['): #if a double header
            desc=desc.strip('[')
            desc=desc.strip(']')
            possibles.append(desc) #list it as a possible
    tests=[]
    for j in range(len(possibles)):
        testCase=possibles[j].split('_')[0] #get the test name
        if (testCase in template['Models and Tests to Run']['test_cases']):
            tests.append(possibles[j])
    return tests

def genWidgets(arr,template):
    #set up types and box
    types = typeWidgets(arr)
    box = []
    tests = getTests(arr,template)
    for i in range(len(types)):
        desc = arr[i].split('=')[0].strip()
        desc = desc.strip("[")
        desc = desc.strip("]")
        found = True
        if(desc not in template.keys()):
            found = False
            for k in template.keys():
                if(desc in template[k].keys()):
                    found = True
                else:
                    try:
                        for l in template[k].keys():
                            if(desc in template[k][l].keys() or desc == "Initial"):
                                found = True
                    except: AttributeError
            if not found:
                print("Possible key error: " + desc + " not recognized")
        #create appropriate widget
        if types[i] == 'String':
            widg = makeStringWidget(arr[i],template,tests)
        elif types[i] == 'Float':
            widg = makeFloatWidget(arr[i])
        elif types[i] == 'Int':
            widg = makeIntWidget(arr[i])
        elif types[i] == 'Label':
            widg = widgets.Label(value=arr[i],layout = Layout(align = 'center',height = '60px',font_size='24px'))
        else: #type is a bool
            widg = makeBoolWidget(arr[i])
        #add widget to box
        box.append(widg)
    return(VBox(box))

def loadConf(filename):
    arr = importArray(filename)
    template = ConfigFileConstructor("meeting.conf").get_config_template()
    return genWidgets(arr,template)

runButton = widgets.Button(description='Run Test')
inFile = widgets.Text(placeholder = 'enter input file path',val='',disabled = False)
runBox = HBox([inFile,runButton])

output = widgets.Output()

@output.capture()
def on_runButton_clicked(b):
    from IPython.core.display import display, HTML
    #display(HTML("<style>.container { width:100% !important; }</style>"))
    conFile = runBox.children[0].get_state()['value']
    cwd = os.getcwd()
    in_path = os.path.dirname(os.path.realpath(conFile))
    os.chdir(in_path)
    fileName = conFile.split('/')[-1]
    mastml = MASTML.MASTMLDriver(configfile=fileName)
    mastml.run_MASTML()
    os.chdir(cwd)

runButton.on_click(on_runButton_clicked)

saveButton = widgets.Button(description='Save to File')
outFile = widgets.Text(placeholder = 'enter output file path', val='', disabled = False)
saveBox= HBox([outFile,saveButton])

@output.capture()
def on_saveButton_clicked(b):
    conFile = saveBox.children[0].get_state()['value']
    allWidgets = IOBox.children[1]
    exportBox(allWidgets, conFile)
    print("File saved!")

saveButton.on_click(on_saveButton_clicked)

loadButton = widgets.Button(description='Load config file')
initConf = widgets.Text(placeholder = 'enter initial configuration file path', val='', disabled = False)
loadBox = HBox([initConf,loadButton])

IOBox = VBox([loadBox,saveBox,runBox])

@output.capture()
def on_loadButton_clicked(b):
    os.chdir(home)
    conFile = loadBox.children[0].get_state()['value']
    if(conFile != ''):
        from IPython.display import display
        global IOBox
        IOBox.close()
        IOBox = VBox([loadBox,saveBox,runBox])
        allWidgets = loadConf(conFile)
        IOBox.children = (IOBox.children[0],allWidgets,IOBox.children[1],IOBox.children[2])
        display(IOBox)
    
loadButton.on_click(on_loadButton_clicked)
display(IOBox)
display(output)

Box(children=(HBox(children=(Text(value='', placeholder='enter initial configuration file path'), Button(descr…

Output()